TODO Luki format??

Co prioritizovat? https://github.com/zdenkas/human-tafazzin-variants-database/blob/main/TODO.txt
Konzistence cisel (pozic) a nukleotidu
Jake k tomu pouzit data?
Kolik nekonzistenci tam najdeme? Pak to poslat BSF


In [1]:
import pandas as pd
from Bio import SeqIO

In [2]:
import helpers

In [3]:
database_folder = '../database_versions'

In [4]:
input_path_prefix = '../database_versions/0001_2023-10-30-21-51-49-628455_Human-TAFAZZIN-Variants-Database_' 
# TODO get this automatically in next notebooks

output_path_prefix = helpers.create_database_output_path_prefix()

version number: 0003
database output path prefix: ../database_versions/0003_2024-01-11-17-35-08-949032_Human-TAFAZZIN-Variants-Database_


In [5]:
taz_cdna_path = '../data_external/Homo_sapiens_ENST00000601016_6_sequence.fa'
taz_positions_path = '../data_external/ExonsSpreadsheet-Homo_sapiens_Transcript_Exons_ENST00000601016_whole_table.csv'

### Load positions

in hg38

In [6]:
taz_positions = pd.read_csv(taz_positions_path)

In [7]:
taz_positions['Start'] = pd.to_numeric(taz_positions['Start'].str.replace(',', ''), errors='coerce')
taz_positions['Start'] = taz_positions['Start'].fillna(0).astype(int)

In [8]:
taz_positions['End'] = pd.to_numeric(taz_positions['End'].str.replace(',', ''), errors='coerce')
taz_positions['End'] = taz_positions['End'].fillna(0).astype(int)

In [9]:
taz_positions.head()

,No.,Exon / Intron,Start,End,Start Phase,End Phase,Length,Sequence
0,NaN,5' upstream sequence,0,0,NaN,NaN,NaN,"{""provisional"":{..."
1,1,ENSE00003022330,154411539,154411952,-,1,414,"{""url"":""/Homo_sa..."
2,,Intron 1-2,154411953,154412085,NaN,NaN,133,"{""provisional"":{..."
3,2,ENSE00003127263,154412086,154412214,1,1,129,"{""provisional"":{..."
4,,Intron 2-3,154412215,154413206,NaN,NaN,992,"{""provisional"":{..."


### Load cdna

In [10]:
for seq_record in SeqIO.parse(taz_cdna_path, "fasta"):
    print(seq_record.id)
    print(seq_record.name)
    print(seq_record.description)
    print(repr(seq_record.seq))
    print(len(seq_record))
    print('-----')

ENST00000601016.6
ENST00000601016.6
ENST00000601016.6 TAFAZZIN-213 cdna:protein_coding
Seq('GCTTTCCGGCGGTTGCACCGGGCCGGGGTGCCAGCGCCCGCCTTCCCGTTTCCT...AAA')
1906
-----
TAFAZZIN-213
TAFAZZIN-213
TAFAZZIN-213 cds:protein_coding
Seq('ATGCCTCTGCACGTGAAGTGGCCGTTCCCCGCGGTGCCGCCGCTCACCTGGACC...TAG')
879
-----
TAFAZZIN-213
TAFAZZIN-213
TAFAZZIN-213 ENSE00003022330 exon:protein_coding
Seq('GCTTTCCGGCGGTTGCACCGGGCCGGGGTGCCAGCGCCCGCCTTCCCGTTTCCT...CCA')
414
-----
TAFAZZIN-213
TAFAZZIN-213
TAFAZZIN-213 ENSE00003127263 exon:protein_coding
Seq('AGTACATGAACCACCTGACCGTGCACAACAGGGAGGTGCTGTACGAGCTCATCG...GGG')
129
-----
TAFAZZIN-213
TAFAZZIN-213
TAFAZZIN-213 ENSE00003736420 exon:protein_coding
Seq('GGATCCTGAAACTCCGCCACATCTGGAACCTGAAGTTGATGCGTTG')
46
-----
TAFAZZIN-213
TAFAZZIN-213
TAFAZZIN-213 ENSE00003017673 exon:protein_coding
Seq('GACCCCTGCAGCTGCAGACATCTGCTTCACCAAGGAGCTACACTCCCACTTCTT...GAG')
86
-----
TAFAZZIN-213
TAFAZZIN-213
TAFAZZIN-213 ENSE00003050674 exon:protein_coding
Seq('GAGCAGAATTTTTCCAAGCAGA

In [11]:
len(seq_record)

10188

In [12]:
seq_record.seq

Seq('GCTTTCCGGCGGTTGCACCGGGCCGGGGTGCCAGCGCCCGCCTTCCCGTTTCCT...AAA')

In [13]:
seq_record.seq[-1]

'A'

In [14]:
seq_record

SeqRecord(seq=Seq('GCTTTCCGGCGGTTGCACCGGGCCGGGGTGCCAGCGCCCGCCTTCCCGTTTCCT...AAA'), id='X', name='X', description='X dna:chromosome chromosome:GRCh38:X:154411539:154421726:1', dbxrefs=[])

### Load Tafazzin database

In [15]:
df_pathogenic = pd.read_csv(input_path_prefix + 'pathogenic.csv')
df_vus = pd.read_csv(input_path_prefix + 'vus.csv')
df_exon5 = pd.read_csv(input_path_prefix + 'exon5.csv')
df_benign = pd.read_csv(input_path_prefix + 'benign.csv')

In [16]:
df_pathogenic.head(1)

,Location,Location in Genome release 37 (hg19),Location in Genome release 38 (hg38),Protein Variant Type,Impact of Variant,DNA Modifications,Protein or mRNA Variants,Functional outcome (MLCL/CL ratio),Taffazin Functional motifs,Method of Validation,References,Source,Additional variants in other genes,Location and Order of Discovery,Notes,Unnamed: 15
0,Exon 1,X:153640189,X:154411852,Frameshift,NaN,c.9_10dupG,p.His4Alafs*130,MLCL/CL elevated,NaN,NaN,Ref. 1 (Pat.1); Ref. 80; Ref. 113,NaN,NaN,1-1,NaN,NaN


In [17]:
loc_38_col = 'Location in Genome release 38 (hg38)'
loc_37_col = 'Location in Genome release 37 (hg19)'
dna_modif_col = 'DNA Modifications'

In [26]:
for key, row in df_pathogenic.iterrows():
    taz_database_38_loc = int(row[loc_38_col].split(':')[1])
    taz_database_37_loc = int(row[loc_37_col].split(':')[1])
    taz_database_dna_modif = row[dna_modif_col]
    break

In [28]:
def get_exon_intron_from_taz_positions(taz_database_38_loc, taz_positions=taz_positions):
    matching_rows = taz_positions[(taz_positions['Start'] <= taz_database_38_loc) & (taz_positions['End'] >= taz_database_38_loc)]
    exon_intron_values = matching_rows['Exon / Intron'].values

    # If you expect only one match and want to get a single value
    if len(exon_intron_values) == 1:
        return exon_intron_values[0]
    else:
        print("Multiple or no matching rows found!")
        return None

In [29]:
get_exon_intron_from_taz_positions(taz_database_38_loc)

'ENSE00003022330'

In [24]:
taz_positions

,No.,Exon / Intron,Start,End,Start Phase,End Phase,Length,Sequence
0,NaN,5' upstream sequence,0,0,NaN,NaN,NaN,"{""provisional"":{..."
1,1,ENSE00003022330,154411539,154411952,-,1,414,"{""url"":""/Homo_sa..."
2,,Intron 1-2,154411953,154412085,NaN,NaN,133,"{""provisional"":{..."
3,2,ENSE00003127263,154412086,154412214,1,1,129,"{""provisional"":{..."
4,,Intron 2-3,154412215,154413206,NaN,NaN,992,"{""provisional"":{..."
5,3,ENSE00003736420,154413207,154413252,1,2,46,"{""url"":""/Homo_sa..."
6,,Intron 3-4,154413253,154413481,NaN,NaN,229,"{""provisional"":{..."
7,4,ENSE00003017673,154413482,154413567,2,1,86,"{""provisional"":{..."
8,,Intron 4-5,154413568,154414100,NaN,NaN,533,"{""url"":""/Homo_sa..."
9,5,ENSE00003050674,154414101,154414190,1,1,90,"{""provisional"":{..."


In [19]:
taz_positions.Start

0             0
1     154411539
2     154411953
3     154412086
4     154412215
5     154413207
6     154413253
7     154413482
8     154413568
9     154414101
10    154414191
11    154419543
12    154419624
13    154419705
14    154419747
15    154420032
16    154420095
17    154420212
18    154420265
19    154420658
20    154420736
21    154420903
22            0
Name: Start, dtype: int64

In [ ]:
taz_positions.head(20)

In [20]:
taz_database_38

154411852

In [21]:
taz_database_37

153640189

In [22]:
taz_database_dna_modif

'c.9_10dupG'

In [23]:
df_pathogenic.Location.value_counts()

Exon 8                 61
Exon 2                 53
Exon 3                 41
Exon 4                 39
Exon 10                33
Exon 6                 31
Large Deletion         30
Exon 7                 14
Exon 11                14
Exon 9                 14
Intron 1               12
Intron 10              10
Exon 1                  9
Intron 3                8
Intron 6                7
Intron 5                7
Intron 7                6
Intron 9                6
Intron 2                5
Intron 4                2
No Mutation             2
Intron 8                1
Chromosomal Changes     1
Name: Location, dtype: int64